In [36]:
import sympy as sp

Euler-Lagrange equation:

$\frac{\partial}{\partial t} (\frac{\partial L}{\partial \dot q})= \frac{\partial L}{\partial q}$

In [37]:
t, v = sp.symbols('t v', real=True)
r = sp.Symbol('r', real=True, positive=True)
psi, gamma = sp.symbols('\\psi \\gamma', real=True)
r_dot, psi_dot, gamma_dot = sp.symbols('\\dot{r} \\dot{\\psi} \\dot{\\gamma}', real=True)
r_ddot = sp.symbols('\\ddot{r}', real=True)
M, c, e = sp.symbols('M c e', positive = True)

p_psi, r_0, v_0, theta_rv = sp.symbols('p_psi r_0 v_0 \\theta_rv', real=True)

U_e = sp.Function('U')(r)

r_t = sp.Function('r')(t)
psi_t = sp.Function('psi')(t)
gamma_t = sp.Function('gamma')(t)
U_e_t = sp.Function('p')(r_t)

dr_dt = r_t.diff(t)
dpsi_dt = psi_t.diff(t)
dgamma_dt = gamma_t.diff(t)

In [38]:
L = -M * c ** 2 / gamma - U_e

## Developing $\ddot{r}$ equation
the Euler-Lagrange equation is: $\frac{\partial}{\partial t} (\frac{\partial L}{\partial \dot r})= \frac{\partial L}{\partial r}$


In [39]:
v_squared = r_dot ** 2 + r ** 2 * psi_dot ** 2
f_gamma = (1 / sp.sqrt(1 - v ** 2 / c ** 2))

Notice that by differentiation by chain rule $\frac{\partial L}{\partial \dot r} = \frac{\partial L}{\partial \gamma} \frac{\partial \gamma}{\partial v} \frac{\partial v}{\partial \dot{r}} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial \dot{r}} $

In [40]:
dL_dgamma = L.diff(gamma)
dL_dgamma = dL_dgamma
dL_dgamma

M*c**2/\gamma**2

In [41]:
dgamma_dv = f_gamma.diff(v)
dgamma_dv = dgamma_dv.subs(f_gamma, gamma)
dgamma_dv

\gamma**3*v/c**2

In [42]:
dv_drdot = sp.sqrt(v_squared).diff(r_dot)
dv_drdot = dv_drdot.subs(sp.sqrt(v_squared), v)
dv_drdot

\dot{r}/v

In [43]:
dL_drdot = dL_dgamma * dgamma_dv * dv_drdot
dL_drdot = dL_drdot.simplify()
dL_drdot

M*\dot{r}*\gamma

In [44]:
dL_drdot_t = dL_drdot.subs({gamma: gamma_t, r_dot: dr_dt})
dL_drdot_t = dL_drdot_t.simplify()
dL_drdot_t

M*gamma(t)*Derivative(r(t), t)

And here is the left side of the Euler-Lagrange equation:

In [45]:
d_dt_dL_drdot = dL_drdot_t.diff(t)
d_dt_dL_drdot

M*gamma(t)*Derivative(r(t), (t, 2)) + M*Derivative(gamma(t), t)*Derivative(r(t), t)

Now lets start with the right side of the Euler-Lagrange equation $\frac{\partial L}{\partial r}$,
Notice that by differentiation by chain rule:

$\frac{\partial L}{\partial r} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial r} - e ^ 2  / r ^ 2$

Notice that $\frac{\partial L}{\partial v} = \frac{\partial L}{\partial \gamma} \frac{\partial \gamma}{\partial v}$

In [46]:
dL_dv = dL_dgamma * dgamma_dv
dL_dv = dL_dv.simplify()
dL_dv

M*\gamma*v

In [47]:
dv_dr = sp.sqrt(v_squared).diff(r)
dv_dr = dv_dr.subs(sp.sqrt(v_squared), v)
dv_dr

\dot{\psi}**2*r/v

In [48]:
dL_dr = dL_dv * dv_dr + U_e.diff(r)
dL_dr = dL_dr.simplify()
dL_dr

M*\dot{\psi}**2*\gamma*r + Derivative(U(r), r)

In [49]:
r_ddot_eq = d_dt_dL_drdot - dL_dr.subs({r: r_t, gamma: gamma_t, psi_dot: psi_t.diff(t)})
r_ddot_eq = (r_ddot_eq / M / gamma_t).simplify()
r_ddot_eq.simplify()

-r(t)*Derivative(psi(t), t)**2 + Derivative(r(t), (t, 2)) + Derivative(gamma(t), t)*Derivative(r(t), t)/gamma(t) - Derivative(U(r(t)), r(t))/(M*gamma(t))

### Developing $\dot{\phi}$ equation
Lets find what $\dot{\phi}$ equals to, using the Euler-Lagrange equation for $\phi$:

Notice that:
$\frac{\partial L}{\partial \dot{\phi}} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial \dot{\phi}}$

In [50]:
dv_dpsidot = sp.sqrt(v_squared).diff(psi_dot)
dv_dpsidot = dv_dpsidot.subs(sp.sqrt(v_squared), v)
dv_dpsidot

\dot{\psi}*r**2/v

In [51]:
dL_dpsidot = dL_dv * dv_dpsidot
dL_dpsidot

M*\dot{\psi}*\gamma*r**2

From the Euler-Lagrange equation: $\frac{\partial}{\partial t} (\frac{\partial L}{\partial \dot{\psi}}) = 0 \to \gamma M r^2 \dot{\psi} = p_{\psi} \to \dot{\psi} = \frac{p_{\psi}}{\gamma M r^2}$

In [52]:
v_t = sp.Function('v')(t)

In [53]:
f_gamma_t = f_gamma.subs(v, v_t)
dgamma_dt = f_gamma_t.diff(t)
dgamma_dt = dgamma_dt.subs(f_gamma_t, gamma_t) 
dgamma_dt

gamma(t)**3*v(t)*Derivative(v(t), t)/c**2

In [54]:
gamma_eq = f_gamma.subs({v: sp.sqrt(v_squared)})
psi_dot_eq = p_psi / gamma / M / r ** 2
psi_dot_eq_subs = psi_dot_eq.subs({gamma: gamma_eq})
psi_dot_squared_eq = psi_dot_eq_subs ** 2
psi_dot_s = sp.solve(psi_dot_squared_eq - psi_dot ** 2, psi_dot)[1]
psi_dot_s_squared = psi_dot_s ** 2
psi_dot_s_squared

p_psi**2*(-\dot{r}**2 + c**2)/(r**2*(M**2*c**2*r**2 + p_psi**2))

In [55]:
psi_dot_s_t = psi_dot_s.subs({r_dot: dr_dt, r: r_t, gamma: gamma_t})
psi_dot_s_t

p_psi*sqrt(c**2 - Derivative(r(t), t)**2)/(sqrt(M**2*c**2*r(t)**2 + p_psi**2)*r(t))

In [56]:
f_v_t = sp.sqrt(v_squared).subs({r_dot: dr_dt, r: r_t, psi_dot: psi_dot_s_t})
dv_dt = f_v_t.diff(t).subs(f_v_t, v_t)
dv_dt

(-M**2*c**2*p_psi**2*(c**2 - Derivative(r(t), t)**2)*r(t)*Derivative(r(t), t)/(M**2*c**2*r(t)**2 + p_psi**2)**2 - p_psi**2*Derivative(r(t), t)*Derivative(r(t), (t, 2))/(M**2*c**2*r(t)**2 + p_psi**2) + Derivative(r(t), t)*Derivative(r(t), (t, 2)))/v(t)

In [57]:
dgamma_dt_subs = dgamma_dt.subs(v_t.diff(t), dv_dt)
dgamma_dt_subs

(-M**2*c**2*p_psi**2*(c**2 - Derivative(r(t), t)**2)*r(t)*Derivative(r(t), t)/(M**2*c**2*r(t)**2 + p_psi**2)**2 - p_psi**2*Derivative(r(t), t)*Derivative(r(t), (t, 2))/(M**2*c**2*r(t)**2 + p_psi**2) + Derivative(r(t), t)*Derivative(r(t), (t, 2)))*gamma(t)**3/c**2

In [58]:
f_gamma_v = f_gamma.subs(v**2, v_squared).simplify().subs(psi_dot, psi_dot_s).simplify().subs({r_dot: dr_dt, r: r_t, gamma: gamma_t})
gamma_eq_0 = sp.Eq(gamma, f_gamma_v)
gamma_eq_0 = sp.Eq(gamma_eq_0.lhs ** 2, gamma_eq_0.rhs ** 2)
gamma_eq_0 = sp.Eq(1 / gamma_eq_0.lhs, 1 / gamma_eq_0.rhs)
gamma_eq_0 = sp.Eq(gamma_eq_0.lhs / r_t, gamma_eq_0.rhs / r_t)
gamma_eq_0 = gamma_eq_0.simplify()
gamma_eq_0

Eq(1/(\gamma**2*r(t)), M**2*(c**2 - Derivative(r(t), t)**2)*r(t)/(M**2*c**2*r(t)**2 + p_psi**2))

In [59]:
dgamma_dt_subs = dgamma_dt_subs.subs(gamma_eq_0.rhs, gamma_eq_0.lhs).simplify()
dgamma_dt_subs

(M**2*\gamma**2*r(t)**3*Derivative(r(t), (t, 2)) - p_psi**2)*gamma(t)**3*Derivative(r(t), t)/(\gamma**2*(M**2*c**2*r(t)**2 + p_psi**2)*r(t))

In [60]:
f_gamma_simplified = f_gamma.subs(v, sp.sqrt(v_squared))
f_gamma_simplified = f_gamma_simplified.subs(psi_dot, psi_dot_s).simplify() 

replacement_a1 = sp.sqrt(M**2 * c**2 * r**2 + (p_psi**2))
replacement_a2 = M * c * r * sp.sqrt(1 + (p_psi / M / c / r) ** 2)

f_gamma_simplified = f_gamma_simplified.replace(replacement_a1, replacement_a2)

replacement_b1 = 1 / sp.sqrt(-r_dot**2 + c**2)
replacement_b2 = 1 / (c * sp.sqrt(1 - r_dot**2 / c**2))

f_gamma_simplified = f_gamma_simplified.replace(replacement_b1, replacement_b2)

f_gamma_simplified

sqrt(1 + p_psi**2/(M**2*c**2*r**2))/sqrt(-\dot{r}**2/c**2 + 1)

Ensure my simplification is correct:

In [61]:
f_gamma_simplified.simplify() - f_gamma.subs(v, sp.sqrt(v_squared)).simplify().subs(psi_dot, psi_dot_s).simplify()

0

In [62]:
gamma_eq_ = sp.Eq(gamma, f_gamma_simplified)
gamma_eq_ = sp.Eq(gamma_eq_.lhs**2, gamma_eq_.rhs**2)
gamma_eq_ = sp.Eq(1 / gamma_eq_.lhs, 1 / gamma_eq_.rhs)
denominator = -(r_dot / c) ** 2 + 1
gamma_eq_ = sp.Eq(gamma_eq_.lhs / denominator, gamma_eq_.rhs / denominator)
gamma_eq_

Eq(1/(\gamma**2*(-\dot{r}**2/c**2 + 1)), 1/(1 + p_psi**2/(M**2*c**2*r**2)))

In [63]:
dgamma_dt_simplified = dgamma_dt_subs.subs({
    dr_dt.diff(t): r_ddot, 
    dr_dt: r_dot, 
    r_t: r, 
    gamma_t: gamma
})

replacement_a1 = M ** 2 * r_ddot * gamma ** 2 * r ** 3 - p_psi ** 2
replacement_a2 = M ** 2 * r ** 3 * c ** 2 * (r_ddot * gamma ** 2 / c ** 2 - p_psi ** 2 / M ** 2 / r ** 3 / c ** 2)

dgamma_dt_simplified = dgamma_dt_simplified.replace(replacement_a1, replacement_a2)

replacement_b3 = 1 / (M ** 2 * c ** 2 * r ** 2 + p_psi ** 2)
replacement_b4 = 1 / ((M ** 2 * c ** 2 * r ** 2) * (1 + p_psi ** 2 / M ** 2 / c ** 2 / r ** 2))

dgamma_dt_simplified = dgamma_dt_simplified.replace(replacement_b3, replacement_b4)
dgamma_dt_simplified

\dot{r}*\gamma*(\ddot{r}*\gamma**2/c**2 - p_psi**2/(M**2*c**2*r**3))/(1 + p_psi**2/(M**2*c**2*r**2))

Ensure my simplification is correct:

In [64]:
dgamma_dt_simplified.simplify() - dgamma_dt_subs.subs({
    dr_dt.diff(t): r_ddot, 
    dr_dt: r_dot, 
    r_t: r, 
    gamma_t: gamma
}).simplify()

0

In [65]:
dgamma_dt_simplified = dgamma_dt_simplified.replace(gamma_eq_.rhs, gamma_eq_.lhs)
dgamma_dt_simplified

\dot{r}*(\ddot{r}*\gamma**2/c**2 - p_psi**2/(M**2*c**2*r**3))/(\gamma*(-\dot{r}**2/c**2 + 1))

In [66]:
r_ddot_eq = r_ddot_eq.subs({dr_dt.diff(t): r_ddot, dr_dt: r_dot, r_t: r, gamma_t.diff(t): gamma_dot, gamma_t: gamma, psi_t.diff(t): psi_dot})
r_ddot_eq = r_ddot_eq.subs({gamma_dot: dgamma_dt_simplified.simplify(), psi_dot: psi_dot_eq}).simplify()
r_ddot_eq

(M**2*\ddot{r}*\gamma**2*c**2*r**3 + M*\dot{r}**2*\gamma*r**3*Derivative(U(r), r) - M*\gamma*c**2*r**3*Derivative(U(r), r) - c**2*p_psi**2)/(M**2*\gamma**2*r**3*(-\dot{r}**2 + c**2))

In [67]:
r_ddot_s = sp.solve(r_ddot_eq, r_ddot)[0]
r_ddot_s

-\dot{r}**2*Derivative(U(r), r)/(M*\gamma*c**2) + Derivative(U(r), r)/(M*\gamma) + p_psi**2/(M**2*\gamma**2*r**3)

## Simplifying the equations

### Simplifying $\ddot{r}$ equation

In [68]:
r_ddot_s

-\dot{r}**2*Derivative(U(r), r)/(M*\gamma*c**2) + Derivative(U(r), r)/(M*\gamma) + p_psi**2/(M**2*\gamma**2*r**3)

In [69]:
r_ddot_s_0 = r_ddot_s.collect(U_e.diff(r)).collect(1 / M / gamma)
r_ddot_s_0

(-\dot{r}**2/c**2 + 1)*Derivative(U(r), r)/(M*\gamma) + p_psi**2/(M**2*\gamma**2*r**3)

## In Summary the equations of motion:
- $\vec{p_{\psi}} = M \vec{r_0} \times \vec{v_0} \to p_{\psi} = M |\vec{r_0}| |\vec{v_0}| sin(\theta)$
- $\ddot r =  \frac{p_{\psi}^{2}}{M^{2} \gamma^{2} r^{3}} + \frac{\frac{d}{d r} U{\left(r \right)}} {M \gamma} \left(1 - (\frac{\dot{r}}{c})^2\right)$

- $ \dot \psi = \frac{p_{\psi}}{M \gamma r^{2}}$